In [1]:
import numpy as np
import pandas as pd
from scipy import optimize


In [2]:
path = 'Data/YYL_data/'
ob09 = pd.read_csv(path + 'AAPL_09.04.2018.csv')
ob10 = pd.read_csv(path + 'AAPL_10.04.2018.csv')
ob11 = pd.read_csv(path + 'AAPL_11.04.2018.csv')
ob12 = pd.read_csv(path + 'AAPL_12.04.2018.csv')
ob13 = pd.read_csv(path + 'AAPL_13.04.2018.csv')

ob = pd.concat([ob11, ob12, ob13])
ob[['AskVolume', 'BidVolume']] = ob[['AskVolume', 'BidVolume']].astype(int)
ob[['AskVolume', 'BidVolume']] = ob[['AskVolume', 'BidVolume']].astype(int)

t_ob = ob.copy()
t_ob = t_ob.rename(columns = {'AskVolume': 'Ask Size', 'BidVolume': 'Bid Size', 'Ask': 'Ask Price', 'Bid': 'Bid Price'})
#only consider data in exchange P, could repeat the process for other exchanges
#t_ob = t_ob[(t_ob['Bid Exchange'] == 'P') & (t_ob['Ask Exchange'] == 'P')]
#t_ob = t_ob[(t_ob['Exchange'] == 'Z')]
t_ob['bsize'] = 0
t_ob['asize'] = 0
#remove zero and negative spreads
t_ob = t_ob[t_ob['Ask Price']>t_ob['Bid Price']]
t_ob.index = range(len(t_ob))

n=5
seperator_b = np.percentile(t_ob['Bid Size'], np.arange(0, 100, 100/n))
seperator_a = np.percentile(t_ob['Ask Size'], np.arange(0, 100, 100/n))


def transformOB(t_ob, n): 
    
    # devide the ask queue and bid queue into n bucket, by taking 100/n -th percentile
    # n*n buckets in total
    seperator_b = np.percentile(t_ob['Bid Size'], np.arange(0, 100, 100/n))
    seperator_a = np.percentile(t_ob['Ask Size'], np.arange(0, 100, 100/n))
    
    # normalize the bid size and ask size so that (bid size, ask size) = (i,j) represents ith bucket of bid and jth bucket of ask
    # for each i,j
    # for example, (bid size, ask size) falling into (20%-40%, 40%-60%) range of bid and ask sizes should be convert to (0.4, 0.6) 
    for i in range(n):
        if i == 0:
            t_ob['bsize'][t_ob['Bid Size'] <= seperator_b[i+1]] = (i+1)/n
            t_ob['asize'][t_ob['Ask Size'] <= seperator_a[i+1]] = (i+1)/n
        elif i == n-1:
            t_ob['bsize'][t_ob['Bid Size'] > seperator_b[i]] = (i+1)/n
            t_ob['asize'][t_ob['Ask Size'] > seperator_a[i]] = (i+1)/n
        else:
            t_ob['bsize'][(t_ob['Bid Size'] > seperator_b[i]) & (t_ob['Bid Size'] <= seperator_b[i+1])] = (i+1)/n
            t_ob['asize'][(t_ob['Ask Size'] > seperator_a[i]) & (t_ob['Ask Size'] <= seperator_a[i+1])] = (i+1)/n
    
    return t_ob

def get_count(t_ob):
    return t_ob.groupby(['bsize', 'asize']).size().reset_index(name="Count")

def bucket_dist(t_ob, count, n):
    
    #get the probability of occurrences of (i*n,j*n)-th bucket
    d = np.zeros((n, n))
    
    num_data = np.sum(count['Count'])
    
    for i in range(n):
        for j in range(n):
            count_ij = count['Count'][(count['bsize'] == (i+1)/n) & (count['asize'] == (j+1)/n)]
            if len(count_ij) == 0:
                d[i,j] = 0
            else:
                d[i,j] = count['Count'][(count['bsize'] == (i+1)/n) & (count['asize'] == (j+1)/n)] / num_data
    
    return d


def get_price_change(t_ob):
    change_idx = []
    
    #get the midprice
    t_ob['midprice'] = (t_ob['Bid Price'] + t_ob['Ask Price'])/2
    
    #get the index of price-change moment
    for k in range(1, len(t_ob['midprice'])):
        
        if t_ob['midprice'][k] != t_ob['midprice'][k-1]:
            change_idx.append(k)
            
    return change_idx


def get_up_count(t_ob, idx):
    t_ob['up'] = 0
    
    #get the index of price-up moment
    current_idx = 0
    for i in idx:
        # mark all the previous moments starting from the previous price-change index to 'up = 1'
        if t_ob['midprice'][i] > t_ob['midprice'][i-1]:
            t_ob['up'][current_idx:i] = 1
        current_idx = i
    
    # get the count of each (i,j) & up
    up_count = t_ob.groupby(['bsize', 'asize'])['up'].sum().reset_index(name="Up_Count")
    return up_count


def empirical_prob_up(t_ob,count,up_count,n):
    
    u = np.zeros((n, n))
    
    # get the empiricial probability of events for each (i,j) and price-up
    
    #up_count['prob'] = up_count['Up_Count']/count['Count']
    
    for i in range(n):
        for j in range(n):
            upcount_ij = up_count['Up_Count'][(up_count['bsize'] == (i+1)/n) & (up_count['asize'] == (j+1)/n)]
            count_ij = count['Count'][(count['bsize'] == (i+1)/n) & (count['asize'] == (j+1)/n)]
            if len(count_ij) == 0:
                u[i,j] = 0.5
            else:
                u[i,j] = upcount_ij/count_ij
            #u[i,j] = up_count['prob'][(up_count['bsize'] == (i+1)/n) & (up_count['asize'] == (j+1)/n)]
    
    return u  

#Divide the bid queue and ask queue into 5 buckets respectively
n = 5
t_ob= transformOB(t_ob,n)

count=get_count(t_ob)

D = bucket_dist(t_ob,count,n)
print(D)
np.sum(D)

change_idx = get_price_change(t_ob)
up_count = get_up_count(t_ob, change_idx)

#get the empricial probability of price-up for each busket
U = empirical_prob_up(t_ob,count,up_count,n)
print(U)

# Least square method to find the H
def f(H):
    res = 0
    for i in range(n):
        for j in range(n):
            ii = (i+1)/n
            jj = (j+1)/n
            res += (U[i][j] - (ii+H)/(ii+jj+2*H))**2 * D[i][j]
    return res

result = optimize.minimize(f, 0.5, method='nelder-mead',
               options={'xtol': 1e-8, 'disp': True})

H = result.x[0]

# gradient descent, the same as using a solver
# def error(H,u,n):
#     u_new = np.zeros((n,n))
#     for i in range(n):
#         for j in range(n):
#             ii = (i+1)/n
#             jj = (j+1)/n
#             u_new[i,j] = u[i,j] - (ii + H)/(ii + jj + 2*H)
    
#     return np.sum(u_new**2 *D)
# def grad(H,u):
#     res = 0
#     for i in range(n):
#         for j in range(n):
#             ii = (i+1)/n
#             jj = (j+1)/n
#             res += 2*(u[i,j]-(ii+H)/(ii+jj+2*H))*((ii-jj)/(ii+jj+2*H))*D[i,j]
            
#     return res
# u_copy = U.copy()

# eta=1
# H0=0.5
# T=100000
# H=H0
# print(error(H0,u_copy,n))
# for t in range(T):
#     if (t % 100 ==0):
#         print(t,error(H,u_copy,n))
#     H = H - eta * grad(H,u_copy)
# print(T,error(H,U,n))
# print("H",H)


# compute the model probability of price-up for each busket
model_prob = np.zeros((n, n))
for i in range(n):
    for j in range(n):
        ii = (i+1)/n
        jj = (j+1)/n
        model_prob[i, j] = (ii + H)/(ii + jj + 2*H)

print(model_prob)

C:\Users\kimii\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\kimii\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\kimii\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\kimii\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

[[ 0.06187458  0.02170402  0.04437311  0.06264117  0.01054743]
 [ 0.0621689   0.02033511  0.04730941  0.05897934  0.01033525]
 [ 0.0732502   0.02319612  0.05226486  0.07797292  0.01208745]
 [ 0.04748053  0.017214    0.03444169  0.05441404  0.00940439]
 [ 0.06164187  0.01752201  0.04472902  0.06463977  0.00947283]]


C:\Users\kimii\Anaconda3\lib\site-packages\ipykernel_launcher.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


[[ 0.46050885  0.48375907  0.45210551  0.41400787  0.36794289]
 [ 0.53462512  0.56378324  0.52806713  0.48450737  0.39933775]
 [ 0.57269669  0.59486574  0.57896805  0.51887289  0.4813137 ]
 [ 0.5944933   0.61988072  0.6313593   0.55836478  0.52110626]
 [ 0.56429047  0.65703125  0.57704667  0.52043626  0.49421965]]
Optimization terminated successfully.
         Current function value: 0.001974
         Iterations: 31
         Function evaluations: 65
[[ 0.5         0.46968052  0.4428279   0.41887966  0.39738879]
 [ 0.53031948  0.5         0.47295989  0.4486944   0.4267973 ]
 [ 0.5571721   0.52704011  0.5         0.4755991   0.453469  ]
 [ 0.58112034  0.5513056   0.5244009   0.5         0.47776893]
 [ 0.60261121  0.5732027   0.546531    0.52223107  0.5       ]]


In [ ]:
model_prob = \
np.array([[ 0.5       ,  0.46968052,  0.4428279 ,  0.41887966,  0.39738879],
       [ 0.53031948,  0.5       ,  0.47295989,  0.4486944 ,  0.4267973 ],
       [ 0.5571721 ,  0.52704011,  0.5       ,  0.4755991 ,  0.453469  ],
       [ 0.58112034,  0.5513056 ,  0.5244009 ,  0.5       ,  0.47776893],
       [ 0.60261121,  0.5732027 ,  0.546531  ,  0.52223107,  0.5       ]])

In [14]:
#convert the model probability matrix into a signal
def get_hl_signal(quote_bk, trade_bk, sp):
    res = 0
    #read the current ask size and bid size from the quote book
    bidsize = quote_bk.iloc[-1]['Bid_Size']
    asksize = quote_bk.iloc[-1]['Ask_Size']
    #locate the busket of the bid sizes and ask sizes
    for i in range(n):
        if i == 0:
            if bidsize <= seperator_b[i+1]:
                bsize = (i+1)/n
            if asksize <= seperator_a[i+1]:
                asize = (i+1)/n
        elif i == n-1:
            if bidsize > seperator_b[i]:
                bsize = (i+1)/n
            if asksize > seperator_a[i]:
                asize = (i+1)/n
        else:
            if bidsize <= seperator_b[i+1] and bidsize > seperator_b[i]:
                bsize = (i+1)/n
            if asksize <= seperator_a[i+1] and asksize > seperator_a[i]:
                asize = (i+1)/n
    # get the probability of price-up, if the probability>0.6, return 1, elif it<0.4, return -1, else return 0
    up_prob = model_prob[int(bsize*n - 1), int(asize*n-1)]
    if up_prob > 0.6:
        res = 1
    elif up_prob < 0.4:
        res = -1

    return res
    

In [17]:
# training sample testing
# use the training set to test the accuracy of successfully predicting price-up movement
predict_up = np.zeros((len(t_ob),1))
for i in range(len(t_ob)):
    a = pd.DataFrame()
    a = a.append(t_ob.iloc[i])
    
    signal = get_hl_signal(a,0,0)
    if signal == 1:
        predict_up[i] = 1
        
        
t_ob['predict_up'] = predict_up.astype('int')
#accuracy
print(np.sum(t_ob['predict_up'] == t_ob['up'])/len(t_ob))


0.478029048199
